In [1]:
from elasticsearch import Elasticsearch
url = 'http://192.168.0.201:9200'
es = Elasticsearch(url)

mapping = '''
{
  "mappings": {
    "properties": {
      "city_name": {
        "type": "keyword"
      },
      "population": {
        "type": "integer"
      }
    }
  }
}'''

index_name = 'cities'

index_exists = es.indices.exists(index = index_name)

if not index_exists:
  es.indices.create(index = index_name, body = mapping)

In [3]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

all_cities = []

class city():    
    def __init__(self, city, population):
        self.city = city
        self.population = population


    def add_city_population(self):
        doc['city_name'] = self.city
        doc['population'] = self.population
        try:
            es.index(index=index_name, doc_type="_doc", body=doc)
        except:
            logging.info(str(doc) + ' added in the data!')
        
    
    def update_city_population(self):
        query = {
          "query" : {
            "bool" : {
              "must" : {
                 "term" : {
                    "city_name" : self.city
                 }
              }
            }
          }
        }
        
        q_results = es.search(index=index_name, body=query)
        
        if q_results['hits']['total']['value'] == 0:
            self.add_city_population()
        else:
            for i in range(len(q_results['hits']['hits'])):
                doc_body = q_results['hits']['hits'][i]['_source']
                doc_id = q_results['hits']['hits'][i]['_id']
                doc_body['population'] = self.population
                try:
                    updated_doc = es.update(index=index_name, id=doc_id, body={"doc": doc_body})
                    logging.info(str(doc_body) + " successfully updated!")
                    return updated_doc
                except Exception as e:
                    logging.info(str(e))


    def get_all_cities_population(self):
        query = {
          "query": {
            "match_all": {}
          }
        }
        try:     
            q_results = es.search(index=index_name, body=query)
            
            for i in range(len(q_results['hits']['hits'])):
                doc_data = q_results['hits']['hits'][i]['_source']
                all_cities.append(doc_data)
        except Exception as e:
            logging.error(e)

        return all_cities
        

cp = city('bangalore', 900000)
cp.update_city_population()
cp.get_all_cities_population()